In [0]:
WITH gm_ev_owners AS 
(
WITH gm_ev_owners_by_indiv_id AS 
(
SELECT
  individual_business_id as indiv_id,
  umf_xref_vin_book_mktg_div,  -- Vehicle Brand
  umf_xref_vin_book_brand,  -- Vehicle Nameplate
  ev_type_cd
FROM acquire.vehicle_silver.customer_vin_relation acq
JOIN reporting.vehicle360.fs_veh_ident veh
ON acq.vin = veh.veh_id_nbr
WHERE 1 = 1
AND disposal_dt IS NOT NULL
AND ev_type_cd IS NOT NULL  -- NULL indicates ICE 
)


, indiv_real_id_mapping AS (
SELECT 
a.GM_PERSON_REALID AS gm_person_real_id
, b.INDIV_BUSINESS_ID AS indiv_id
, a.AP006811
, a.AP016727_03
, a.IBE7964B_08
, a.IBE7200
, a.IBE7641_01
, a.AP003248
, a.AP006826
, a.AP003253
, a.AP006241
FROM (SELECT * FROM dataproducts_dev.bronze_acxiom.gm_consumer_list WHERE snapshot = 202508) a
JOIN (SELECT * FROM dataproducts_dev.bronze_acxiom.gm_owners WHERE snapshot = 202508) b
ON a.GM_PERSON_REALID = b.entity_realID_person_id
)

SELECT b.gm_person_real_id
, b.AP006811
, b.AP016727_03
, b.IBE7964B_08
, b.IBE7200
, b.IBE7641_01
, b.AP003248
, b.AP006826
, b.AP003253
, b.AP006241
FROM gm_ev_owners_by_indiv_id a
JOIN indiv_real_id_mapping b
ON a.indiv_id = b.indiv_id
)


-- RZP4VJ, Brad

, pge_utility_audience1 AS 
(
SELECT *
FROM 
gm_ev_owners
WHERE 1 = 1
AND IBE7964B_08 = "San Francisco-Oak-San Jose" -- DMA San Francisco Bay Area
AND AP006811 BETWEEN 1 AND 30 -- The Likelyhood to own a home
-- AND AP016727_03 IN (10, 9) -- The likelihood of a household making specific alternative energy decisions, such as purchasing a home with or installing solar panels.
-- AP006241 IN (1, 2) Use Some Form of Solar Energy
)


, pge_utility_audience2 AS 
(
SELECT *
FROM 
gm_ev_owners
WHERE 1 = 1
AND IBE7964B_08 = "San Francisco-Oak-San Jose" -- DMA San Francisco Bay Area
AND AP006811 BETWEEN 1 AND 30 -- The Likelyhood to own a home
AND (
AP016727_03 IN (10, 9) 
-- The likelihood of a household making specific alternative energy decisions, such as purchasing a home with or installing solar panels.
OR AP006241 IN (1, 2)
) -- Use Some Form of Solar Energy
)


-- , pge_utility_base AS 
-- (
-- SELECT *
-- FROM
-- gm_ev_owners

-- UNION DISTINCT

-- SELECT *
-- FROM 
-- gm_ev_owners
-- WHERE 1 = 1
-- AND AP006811 BETWEEN 1 AND 10 -- The Likely owners of homes
-- AND AP016727_03 IN (10, 9) -- The likelihood of a household making specific alternative energy decisions, such as purchasing a home with or installing solar panels.
-- -- AP006241 IN (1, 2) Use Some Form of Solar Energy
-- )


-- , pge_utility_audience AS 
-- (
-- SELECT DISTINCT gm_person_real_id
-- FROM pge_utility_base
-- WHERE 1 = 1
-- AND IBE7964B_08 = "San Francisco-Oak-San Jose" -- DMA San Francisco Bay Area
-- AND IBE7200 BETWEEN 35 AND 64 -- age 35-64
-- AND IBE7641_01 = 9 -- Household Income: $125K+
-- AND 
-- (
-- AP003248 IN (1, 2) -- Lifestyle: Early tech adopters
-- OR AP006826 BETWEEN 1 AND 10 -- Lifestyle: Green Friendly
-- -- OR AP016727_05 IN (9, 10) -- The likelihood of a household embracing a more conceptual environmentally friendly attitude
-- OR AP003253 IN (1, 2) -- Lifestyle: Likelihood to Purchase Outdoor Power Equipment
-- ) 
-- )

-- , adapter_audience AS
-- (
-- SELECT DISTINCT gm_person_real_id
-- FROM gm_ev_owners

-- UNION DISTINCT
-- (
-- -- SELECT DISTINCT gm_person_real_id
-- -- FROM infact.marsci.pronghorn_audiences_history_detail
-- -- WHERE 1 = 1
-- -- AND Data_YrMonth = '2025-07-01'
-- -- AND audience_name IN
-- -- (
-- -- -- Cadillac
-- -- 'FSP_Cadillac_EVMulti',
-- -- -- 'HPORNM_Cadillac_EscaladeIQ',
-- -- -- 'HPORNM_Cadillac_LYRIQ',
-- -- -- 'HPORNM_Cadillac_OPTIQ',
-- -- -- 'HPORNM_Cadillac_VISTIQ',
-- -- -- 'HP_Cadillac_EVMulti',
-- -- -- 'HP_Cadillac_XTEVMulti',

-- -- -- GMC
-- -- 'FSP_GMC_EVMulti',
-- -- -- 'HPORNM_GMC_HummerEV',
-- -- -- 'HPORNM_GMC_SierraEV',
-- -- -- 'HP_GMC_HummerEV',
-- -- -- 'HP_GMC_SierraEV',

-- -- -- Chevy
-- -- 'FSP_Chevrolet_EVMulti',
-- -- 'FSPDS_Chevrolet_EVMulti'
-- -- -- 'HPORNM_Chevrolet_SilveradoEV',
-- -- -- 'HPORNM_Chevrolet_BlazerEV',
-- -- -- 'HPORNM_Chevrolet_EquinoxEV'
-- -- )

-- SELECT
-- GM_PERSON_REALID AS gm_person_real_id
-- FROM dataproducts_dev.bronze_acxiom.gm_consumer_list
-- WHERE 1 = 1
-- AND snapshot = 202508
-- -- AND PM021206 BETWEEN 1 AND 10 -- Polk - In-Market - Chevrolet EV
-- AND PM020053 BETWEEN 1 AND 10 -- Polk - In-Market - Any EV

-- )
-- )


--  

